In [1]:
import pandas as pd
import numpy as np
import math
import datetime
import itertools
import warnings
warnings.filterwarnings('ignore')

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60


In [2]:
#(inizio e fine compresi)
#Ritorna una lista contenente le chiavi di due città e il tempo di percorrenza tra le due
#nel formato (times, "key1 - key2")
#"key1 - key2" è la chiave
#La lista partirà dalla posizione "inizio" della schedulazione e finità con "fine"
def route(schedule, inizio, fine):
    times = []
    keys = []
    row = dft[get_row(dft, schedule[inizio]).values]
    key1 = ''
    key2 = ''
    impossibile = []
    
    for i in range(inizio+1,fine+1):
        key1 = row.index[0]
        try:
            paese=schedule[i]
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    times1 = times[:]
    return steps_times1

#Data una lista ottenuta da route restituisce la lista dei nomi delle città
def cities_list(serie):
    cities = []
    keys = serie.keys()
    for i in range(0, len(serie)):
        cities = cities + [keys[i].split('-')[0].strip()]
    if len(keys[i].split('-')) > 2: 
        return cities + [keys[i].split('-')[len(keys[i].split('-'))-1].strip()]
    else:
        return cities + [keys[i].split('-')[1].strip()]

In [3]:
#Datogli una schedulazione calcola il tempo di percorrenza
def schedule_time(schedule):
    times = []
    row = dft[get_row(dft, 'Cremona, sesto 39').values]
    keys = []
    key1 = ''
    key2 = ''
    for i in range(1, len(schedule)):
        key1 = row.index[0]
        paese=schedule[i]

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    steps_times = pd.Series(times , keys)
    return sum(steps_times.values)

In [4]:
schedule = pd.Series('Cremona, sesto 39')
schedule = schedule.append(fur1['lunedi'], [1, len(fur1)])
schedule = schedule.append(pd.Series('Ostiano' , [len(schedule)]))
print(schedule)
schedule_time(schedule)

0     Cremona, sesto 39
1              sesto ed
2              grumello
3              SORESINA
4               TRIGOLO
5           CASTELLEONE
6             MONTODINE
7      ripalta cremasca
8               RICENGO
9              SERGNANO
10      CASALE CREMASCO
11             CAMISANO
12             CAPRALBA
13              SONCINO
14        pizzighettone
15              Ostiano
dtype: object


196.0

In [5]:
#mantiene il nodo inizial e il successivo a y invariati, mentre fa il reverse della lista tra i due
#A-b-c-d-E -> A-d-c-b-E
def cross(schedule, x, y):
    i = [schedule[x]]
    j1 = [schedule[y+1]]
    j1_i1 = list(schedule[x+1: y+1].sort_index(ascending = False).values) 
    return i + j1_i1 + j1

In [6]:
#Data una schedulazione e due nodi di cui fare l'incrocio dei nodi
#effettua la tabulazione del nuovo percorso, dopodichè restituisce il migliore
#tra il nuovo e il vecchio
def two_opt(schedule , i, j):
    #computo la prima parte che rimane uguale
    first_part = route(schedule, 0, i)

    #stesso per l'ultima
    final_part = route(schedule, j+1, len(schedule))
    #print(final_part)
    crossed = cross(schedule, i, j)
    central_part = route(crossed, 0,len(crossed))
    
    new_schedule = first_part.append(central_part).append(final_part)
    if(schedule_time(schedule)> sum(new_schedule.values)):
        return new_schedule
    else:
        return route(schedule, 0, len(schedule))

In [7]:
two_opt(schedule, 5, 8)

Cremona, Sesto 39 - Sesto ed Uniti, Roma, 4                              10.0
Sesto ed Uniti, Roma, 4 - Grumello Cremonese ed Uniti, Roma, 2            6.0
Grumello Cremonese ed Uniti, Roma, 2 - Soresina, Giacomo Matteotti, 6    12.0
Soresina, Giacomo Matteotti, 6 - Trigolo, Roma, 26                        9.0
Trigolo, Roma, 26 - Castelleone, Roma, 67                                11.0
Castelleone, Roma, 67 - Montodine,  XXV Aprile, 10                        7.0
Montodine,  XXV Aprile, 10 - Ripalta Cremasca, Roma, 5                    5.0
Ripalta Cremasca, Roma, 5 - Ricengo, Guglielmo Marconi                   12.0
Ricengo, Guglielmo Marconi - Sergnano, IV Novembre,  10                   7.0
Sergnano, IV Novembre,  10 - Casale Cremasco-Vidolasco, Roma, 56          4.0
Casale Cremasco-Vidolasco, Roma, 56 - Camisano, Trieste, 57               6.0
Camisano, Trieste, 57 - Capralba, Piave, 2                               15.0
Capralba, Piave, 2 - Soncino, Dante Alighieri, 23               

In [8]:
#for i in range(1, len(route)-2):
#              for j in range(i+1, len(route)):
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
df = pd.DataFrame({})
for day in week:
    #print(day)
    #Otteniamo la schedulazione unendo Cremona-tabella del giorno-Ostiano
    schedule = pd.Series('Cremona, sesto 39')
    schedule = schedule.append(fur1[day].dropna(), [1, len(fur1)])
    schedule = schedule.append(pd.Series('Ostiano' , [len(schedule)]))
    #print(schedule)
    #Otteniamo la schedulazione filtrando i nan e salviamo anche il suo tempo di percorrenza
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule)
    #Per ogni i e j effettua il 2-opt e salva se il risultato migliora con lo scambio
    #Alla fine, avendo provato a scambiare tutte el coppie e avendole confrontate otteremo la miglior schedulazione
    #per ogni giro di 2-opt parte dalla schedulazione greedy
    for i in range(1, len(schedule)-2):
        for j in range(i+1, len(schedule)-1):
            s = two_opt(schedule, i, j)
            t = sum(s.values)
            if schedule_time(best_schedule)> t:
                best_time = t.copy()
                best_schedule = cities_list(s) 
    df[day] = pd.Series(best_schedule)       
df.to_csv('../Dati/turni_f/2-opt1_shifts.csv', index=False, sep=";")

In [9]:
#for i in range(1, len(route)-2):
#              for j in range(i+1, len(route)):
week= ['martedi', 'mercoledi', 'giovedi']
df = pd.DataFrame({})
for day in week:
    print(day)
    schedule = pd.Series('Cremona, sesto 39')
    schedule = schedule.append(fur2[day].dropna(), [1, len(fur1)])
    schedule = schedule.append(pd.Series('san giovanni in croce' , [len(schedule)]))
    #print(schedule)
    best_schedule = pd.Series(schedule.dropna())
    best_time = schedule_time(schedule)
    for i in range(1, len(schedule)-2):
        for j in range(i+1, len(schedule)-1):
            s = two_opt(schedule, i, j)
            t = sum(s.values)
            if schedule_time(best_schedule)> t:
                best_time = t.copy()
                b_s = s.copy()
                best_schedule = cities_list(s) 
    df[day] = pd.Series(best_schedule)       
df.to_csv('../Dati/turni_f/2-opt2_shifts.csv', index=False, sep=";")

martedi
mercoledi
giovedi
